## Image Preparation

Requires that:
- malign.png be 3-channel
- normal.png be 3-channel
- malign_mask.png be one-channel (BINARY)

In [ ]:
# Required imports
import sys
sys.path.append("../utils")
import os
import cv2
import matplotlib.pyplot as plt
from utils import is_collage_possible, make_collage, make_3_channels_mask, get_latest_model, execute_bash_command, resize_to_dim, get_image_core_name

In [ ]:
# Checks if collage is possible between two images
w, h = is_collage_possible(malign_mask_pth='../pipeline_data_tcia/malign/malign4_mask.png', normal_breast_pth='../pipeline_data_tcia/normal/normal2.png')
w, h

In [ ]:
# If both width and height are different from -1 then collage is possible
if w != -1 and h != -1:
    make_collage(malign_pth='../pipeline_data_tcia/malign/malign4.png', malign_mask_pth='../pipeline_data_tcia/malign/malign4_mask.png', normal_pth='../pipeline_data_tcia/normal/normal2.png', width=w, height=h)

In [ ]:
# Make the collage mask 3-channel
make_3_channels_mask('collage_mask.png', 'collage_mask3.png')
os.remove('collage_mask.png')
os.rename('collage_mask3.png', 'collage_mask.png')

## Basic Harmonisation Training

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/
from evaluate_harmonisation import HarmonisationEvaluator
from evaluate_generation import GenerationEvaluator

In [ ]:
# Normal breast collage Harmonizer creation
! python main_train.py --train_mode harmonization --gpu 0 --train_stages 3 --im_min_size 120 --lrelu_alpha 0.3 --niter 1000 --batch_norm --input_name ../pipeline_data_tcia/normal/normal.png

In [ ]:
# Fine tunes the harmonisation model
m = get_latest_model("TrainedModels/normal")
fine_tune_cmd = "python main_train.py --gpu 0 --train_mode harmonization --input_name ../pipeline_data_tcia/normal/normal.png --naive_img ../notebooks/collage.png --fine_tune --model_dir " + str(m)

for path in execute_bash_command(fine_tune_cmd.split()):
    print(path, end="")

In [ ]:
# Harmonise the naive collage
m = get_latest_model()
harmonise_cmd = "python evaluate_model.py --gpu 0 --model_dir " + str(m) + " --naive_img ../notebooks/collage.png"
for path in execute_bash_command(harmonise_cmd.split()):
        print(path, end="")

In [ ]:
# Evaluate results
base_img = "../pipeline_data_tcia/normal/normal.png"
target_image = os.path.join(get_latest_model("TrainedModels/normal"), "Evaluation_..", "notebooks", "collage.png", "harmonized_w_mask.jpg")
evaluator = GenerationEvaluator(base_img, adjust_sizes=True)

lpips = evaluator.run_lpips_to_image(target_image)
ssim, ms_ssim = evaluator.run_mssim_to_image(target_image)
print(f"LPIPS: {lpips}\nSSIM: {ssim}\nMS-SSIM: {ms_ssim}")

In [ ]:
# Evaluate results according to collage spot
base_img = "../pipeline_data_tcia/normal/normal.png"
target_image = os.path.join(get_latest_model("TrainedModels/normal"), "Evaluation_..", "notebooks", "collage.png", "harmonized_w_mask.jpg")
target_mask = "../notebooks/collage_mask.png"

evaluator = HarmonisationEvaluator(base_img, target_image, target_mask)

lpips = evaluator.run_lpips()
ssim, ms_ssim = evaluator.run_mssim()
evaluator.clean_auxiliary_images()
print(f"LPIPS: {lpips}\nSSIM: {ssim}\nMS-SSIM: {ms_ssim}")

## Combination Harmonisation Testing

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/
from evaluate_harmonisation import HarmonisationEvaluator
from evaluate_generation import GenerationEvaluator

In [ ]:
# Combinations to test in the format below
# (N Stages, Concurrent Stages, Image Min Size, Image Max Size, L.Relu Alpha, N Iterations, Fine-Tune)
# (499, 614) for Standard Definition -> Equivalent to 25% of original normal image
varying_min_combinations = [
    (3, 3, 120, 614, 0.3, 1000, False),
    (3, 3, 120, 614, 0.3, 1000, True),
    (3, 3, 240, 614, 0.3, 1000, False),
    (3, 3, 240, 614, 0.3, 1000, True),
    (3, 3, 480, 614, 0.3, 1000, False),
    (3, 3, 480, 614, 0.3, 1000, True)]

varying_stages_combinations = [
    (6, 3, 120, 614, 0.3, 1000, False),
    (6, 3, 120, 614, 0.3, 1000, True),
    (12, 6, 120, 614, 0.3, 1000, False),
    (12, 6, 120, 614, 0.3, 1000, True)]

In [ ]:
# New combinations to test after first tests above
# (N Stages, Concurrent Stages, Image Min Size, Image Max Size, L.Relu Alpha, N Iterations, Fine-Tune)
# (499, 614) for Standard Definition -> Equivalent to 25% of original normal image
new_combinations = [
    (3, 3, 120, 614, 0.3, 2000, False),
    (6, 3, 120, 614, 0.3, 2000, False),
    (12, 6, 120, 614, 0.3, 2000, False),
    (6, 6, 120, 614, 0.3, 2000, False),
    (12, 10, 120, 614, 0.3, 2000, False),
    (12, 12, 120, 614, 0.3, 2000, False),
    (16, 9, 120, 614, 0.3, 2000, False),
    (16, 12, 120, 614, 0.3, 2000, False),
    (16, 16, 120, 614, 0.3, 2000, False)]

In [ ]:
# New combinations to test after HP improvement
# (N Stages, Concurrent Stages, Image Min Size, Image Max Size, L.Relu Alpha, N Iterations, Fine-Tune)
# (499, 614) for Standard Definition -> Equivalent to 25% of original normal image
new_combinations = [
    (12,6,120,614,0.3,1000,False), (12,10,120,614,0.3,1000,False), (16,9,120,614,0.3,1000,False), (16,12,120,614,0.3,1000,False),
    (20,16,120,614,0.3,1000,False), (22,18,120,614,0.3,1000,False), (24,16,120,614,0.3,1000,False), (24,20,120,614,0.3,1000,False),
    (30,24,120,614,0.3,1000,False)]

In [ ]:
# Execute single experiment with given parameters
def do_single_experiment(naive_image, normal, stages, min_i, max_i, alpha, niter, concurrent, fine_tune):
    
    normal_core_name = get_image_core_name(normal)
    
    cmd = f"python main_train.py --train_mode harmonization --gpu 0 --train_stages {stages} --train_depth {concurrent} --im_min_size {min_i} --im_max_size {max_i} --lrelu_alpha {alpha} --niter {niter} --batch_norm --input_name {normal}"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
        
    if fine_tune:
        m = get_latest_model(f"TrainedModels/{normal_core_name}")
        fine_tune_cmd = f"python main_train.py --gpu 0 --train_mode harmonization --input_name {normal} --naive_img {naive_image} --fine_tune --model_dir " + str(m)
        for path in execute_bash_command(fine_tune_cmd.split()):
            print(path, end="")
        
    m = get_latest_model(f"TrainedModels/{normal_core_name}")
    harmonise_cmd = "python evaluate_model.py --gpu 0 --model_dir " + str(m) + " --naive_img "+ naive_image
    for path in execute_bash_command(harmonise_cmd.split()):
        print(path, end="")
        
    # Evaluate and save to new metrics file
    target_image = os.path.join(get_latest_model(f"TrainedModels/{normal_core_name}"), "Evaluation_..", "notebooks", "collage.png", "harmonized_w_mask.jpg")
    target_mask = "../notebooks/collage_mask.png"
    evaluator = HarmonisationEvaluator(normal, target_image, target_mask)
    lpips = evaluator.run_lpips()
    ssim, ms_ssim = evaluator.run_mssim()
    evaluator.clean_auxiliary_images()
    
    with open('new_evaluation_metrics.txt', 'w') as f:
        print(f"LPIPS: {lpips}\nSSIM: {ssim}\nMS-SSIM: {ms_ssim}", file=f)
        
    # Evaluate and save to generation metrics file
    target_image = os.path.join(get_latest_model(f"TrainedModels/{normal_core_name}"), "Evaluation_..", "notebooks", "collage.png", "harmonized_w_mask.jpg")
    evaluator = GenerationEvaluator(normal, adjust_sizes=True)
    lpips = evaluator.run_lpips_to_image(target_image)
    ssim, ms_ssim = evaluator.run_mssim_to_image(target_image)
    with open('generation_evaluation_metrics.txt', 'w') as f:
        print(f"LPIPS: {lpips}\nSSIM: {ssim}\nMS-SSIM: {ms_ssim}", file=f)
        
    comand = f"zip -r ../(H)S{stages}_MIN{min_i}MAX_{max_i}ITER_{niter}_CONC_{concurrent}_FT_{fine_tune}.zip ."
    for path in execute_bash_command(comand.split()):
        print(path, end="")
        
    ! rm -r mlruns TrainedModels

In [ ]:
# Run experimentations
#for comb in varying_stages_combinations:
#    do_single_experiment('../notebooks/collage.png', '../pipeline_data_tcia/normal/normal.png', comb[0], comb[2], comb[3], comb[4], comb[5], comb[1], comb[6])
    
#for comb in varying_min_combinations:
#    do_single_experiment('../notebooks/collage.png', '../pipeline_data_tcia/normal/normal.png', comb[0], comb[2], comb[3], comb[4], comb[5], comb[1], comb[6])


# New Experiments
for comb in new_combinations:
    do_single_experiment('../notebooks/collage.png', '../pipeline_data_tcia/normal/normal2.png', comb[0], comb[2], comb[3], comb[4], comb[5], comb[1], comb[6])

## Save Model

In [ ]:
# Zip everything from this model
!zip -r current_model.zip .

## Delete Current Model

In [ ]:
# Delete everything that the model produces
! rm -r mlruns TrainedModels
%cd ../notebooks/
! rm collage_mask.png collage.png malign_aux.png normal_aux.png